# Italian Solvency reports Table S.02.01.02; Part 1 Extraction

The scope of this script is to transcribe SFCR table S.02.01.02 for the 18 life insurance companies on the Italian market. The notebook is organized into the following sections:

1) Companies and tables in scope
2) Packages and tools
3) Generation of DataFrames

## Companies in scope

For the year 2024, the companies in scope are the following:

 - Credemvita S.p.A.
 - AXA MPS Assicurazioni Vita
 - CRÈDIT AGRICOLE VITA
 - Società Reale Mutua di Assicurazioni
 - Cardif Vita S.p.A.
 - MEDIOLANUM VITA S.p.A.
 - Generali Italia S.p.A.
 - Banco BPM Vita S.p.A.
 - HDI ASSICURAZIONI S.p.A.
 - Gruppo Assicurativo Poste Vita
 - FIDEURAM VITA S.P.A.
 - CNP Vita Assicura S.p.A.
 - ITAS VITA
 - Helvetia Vita S.p.A.
 - Vittoria Assicurazioni S.p.A.
 - GROUPAMA ASSICURAZIONI S.P.A.
 - UniCredit Allianz Vita S.p.A.
 - Zurich Investments Life S.p.A.

### Solvency and Financial condition reports
According to Article 51 of the Solvency II Directive 2009/138/EC, companies under the regulatory umbrella of EIOPA, companies must publish annually a Solvency and Financial Condition Reports (SFCR) for all legal entities.

Part of the report is mandatory tables that show some financial and actuarial indicators. One such table is S.02.01.02 which shows a simplified balance sheet of the legal entity. This table is inside the scope of this demo.

## Description of the process

The process of extraction is performed in 5 phases:

### Phase 1: Find the reports and identify the relevant tables. 
 1) Identify the new SFCR report and save it into the folder Input.
 2) Identify the pages where the tables of interest are.
 3) Compile the map of the company run in the master_list.csv.

### Phase 2: Run the Extraction script (this script). 
The script performs the following steps (with slight modifications depending on the table format):
 1) Save the page with the table into a separate folder Single_pdf.
 2) Use either a Python package or specialized LLM to create a digital equivalent of the table.
 3) Fix the systemic errors that prevent the table from being saved as DataFrame.
 4) Save the DataFrame into the Output folder.

### Phase 3: Run the Processing script. 
The script applies fixes to the DataFrame to make the numbers closer to the reported numbers. It joins all the tables into a single dataset. 

### Phase 4: Run the Cross-Validation script. 
Applies a series of tests that check for the internal consistency between the numbers. Flags the potential errors.

### Phase 5: Final modifications to the table and a manual inspection. 

## Necessary Python packages

In [1]:
pip install PyPDF2 pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install mistralai

In [3]:
!pip install pycryptodome

## Python packages

In [4]:
import re
import os
import json
import base64
import pdfplumber
import pandas as pd
from pathlib import Path
from PyPDF2 import PdfReader, PdfWriter
from mistralai.models import OCRResponse
from IPython.display import Markdown, display
from mistralai import Mistral, DocumentURLChunk, ImageURLChunk, TextChunk

## API key

The model of choice for this project is Mistral. This was identified as the most economical choice. Additionally, Mistral released a custom model for OCR tasks. 

Useful links:

 - https://docs.mistral.ai/getting-started/quickstart/

 - https://mistral.ai/news/mistral-ocr

In [5]:
api_key = "[YOUR MISTRAL API KEY]"

## Functions

In [6]:
def set_code_index_and_save(table: pd.DataFrame, path: str) -> None:
    """
    Set the 'CODE' column as the index of a DataFrame and save it as a CSV file.
    """
    table = table.set_index("CODE")
    table.to_csv(path)

In [7]:
def extract_tables_from_pdf(pdf_path: str, page_number:int=1) -> pd.DataFrame:
    """
    Extract the first table from a specific page of a PDF and return it as a DataFrame.

    Parameters
    ----------
    pdf_path : str or pathlib.Path
        Path to the PDF file.
    page_number : int, optional (default=1)
        The page number to extract the table from (1-indexed, i.e., first page is 1).

    Returns
    -------
    pd.DataFrame
        A DataFrame containing the extracted table. The first row of the table is
        treated as the header, and the remaining rows as data.

    """
    
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number - 1]  # Pages are zero-indexed
        tables = page.extract_tables()
        df_tables = pd.DataFrame(tables[0][1:], columns=tables[0][0])
        return df_tables

In [8]:
def extract_page(input_pdf_path: str, output_pdf_path: str, page_number: int, password: str = "")-> None:
    """
    Extract a single page from a PDF file and save it as a new PDF.
    """
    
    pdf_reader = PdfReader(input_pdf_path)
    pdf_writer = PdfWriter()

    # Decrypt if necessary
    if pdf_reader.is_encrypted:
        if password:
            pdf_reader.decrypt(password)
        else:
            pdf_reader.decrypt("")  # try empty password
    
    # Add the specified page to the PdfWriter object
    pdf_writer.add_page(pdf_reader.pages[page_number - 1])

    # Write the selected page to a new PDF file
    with open(output_pdf_path, 'wb') as output_pdf_file:
        pdf_writer.write(output_pdf_file)

In [9]:
def encode_pdf(pdf_path: str)-> None:
    """Encode the pdf to base64."""
    try:
        with open(pdf_path, "rb") as pdf_file:
            return base64.b64encode(pdf_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {pdf_path} was not found.")
        return None
    except Exception as e:  # Added general exception handling
        print(f"Error: {e}")
        return None

In [10]:
def markdown_table_to_dataframe(markdown_text, shift_row = 0):
    """
    Convert a Markdown-formatted table into a pandas DataFrame.
    """
    
    # Split the markdown text into lines
    lines = markdown_text.strip().split('\n')

    # Filter out lines that are part of the table
    table_lines = [line for line in lines if line.startswith('|')]

    # Remove the markdown table syntax
    cleaned_lines = [line.strip('|').strip() for line in table_lines]

    # Split each line into columns
    data = [line.split('|') for line in cleaned_lines]

    # Extract headers and rows
    headers = [header.strip() for header in data[1+shift_row] if header.strip()]
    rows = [[cell.strip() for cell in row if cell.strip()] for row in data[3:]]

    # Ensure each row has the same number of columns as the headers
    for row in rows:
        if len(row) < len(headers):
            row += [''] * (len(headers) - len(row))

    # Create a DataFrame
    df = pd.DataFrame(rows, columns=headers)
    return df


In [11]:
def split_strings_to_df(strings):
    """
    Convert a list of strings in the format `"text ..... number"` into a DataFrame.

    Each string is expected to contain a text label followed by dots and a numeric value.
    The numeric value is cleaned (spaces and dots removed) and converted to either
    `int` or `float`. If no numeric value is found, `None` is used instead.

    Parameters
    ----------
    strings : list of str
        A list of strings to be parsed.  
        Example: ["Technical provisions ..... 2.337.991", "Best Estimate ..... 0"]

    Returns
    -------
    pd.DataFrame
        A DataFrame with two columns:
        - "NAME": str, the extracted text (trimmed of dots and whitespace).
        - "C0010": int, float, or None, the extracted numeric value.

    Notes
    -----
    - Numbers with thousand separators like `"2.337.991"` are cleaned into integers (`2337991`).
    - If the number cannot be parsed, it remains as a string.
    - Strings without a numeric value will have `None` in the `"C0010"` column.
    """

    data = []
    for s in strings:
        # Try to match "text ..... number"
        match = re.match(r'^(.*?)\.{2,}\s*([\d\s\.]+)$', s.strip())
        if match:
            left = match.group(1).strip()
            # Clean up number (remove spaces in middle, convert to float/int)
            right = match.group(2).replace(" ", "").replace(".", "")
            if right.isdigit():
                right = int(right)
            else:
                try:
                    right = float(right)
                except:
                    pass
            data.append((left, right))
        else:
            # No number found, just keep text
            data.append((s.strip(), None))

    df = pd.DataFrame(data, columns=["NAME", "C0010"])
    return df

In [12]:
def run_mistral_ocr(output_pdf_path: str, api_key: str)-> str:
    """
    Run OCR on a PDF file using the Mistral OCR API and return the extracted text in Markdown format.

    The function:
    1. Encodes the PDF as a base64 string.
    2. Sends it to the Mistral OCR API.
    3. Extracts the recognized text in Markdown format from the response.
    """
    
    # Getting the base64 string
    base64_pdf = encode_pdf(Path(output_pdf_path))
    
    client = Mistral(api_key=api_key)
    
    ocr_response = client.ocr.process(
        model="mistral-ocr-latest",
        document={
            "type": "document_url",
            "document_url": f"data:application/pdf;base64,{base64_pdf}" 
        },
        include_image_base64=True
    )
    
    # Assuming ocr_response is your instance of OCRResponse
    markdown_texts = [page.markdown for page in ocr_response.pages]
    
    # If you want to concatenate all markdown texts from all pages
    full_markdown_text = "\n".join(markdown_texts)
    
    # Assuming ocr_response is your OCRResponse object
    markdown_text = ocr_response.pages[0].markdown
    return markdown_text
    

In [13]:
def parse_table_to_df(text: str, value_column_name:str) -> pd.DataFrame:
    """
    Parse a long text table into a DataFrame with columns:
    ['code', 'name', 'value'].
    """
    # Split text into lines
    lines = text.splitlines()
    
    rows = []
    buffer = []
    for line in lines:
        # Match lines with pattern: code (Rxxxx) followed by number
        match = re.match(r"^(R\d{4})\s*(.*)$", line)
        if match:
            code = match.group(1)
            rest = match.group(2).strip()
            
            # Check if 'rest' is a number (the value)
            if re.match(r"^[\d\.,]+$", rest):
                name = " ".join(buffer).strip()
                value = rest
                rows.append((code, name, value))
                buffer = []  # reset for next name
            else:
                # It's not just a number, so treat it as name continuation
                buffer.append(line)
        else:
            # If line is just a number after a code line
            if re.match(r"^[\d\.,]+$", line.strip()):
                name = " ".join(buffer).strip()
                value = line.strip()
                # Last buffer entry should contain the code
                code_match = re.search(r"(R\d{4})", buffer[-1]) if buffer else None
                code = code_match.group(1) if code_match else None
                # Remove code from name
                buffer[-1] = buffer[-1].replace(code, "").strip() if code else buffer[-1]
                rows.append((code, " ".join(buffer).strip(), value))
                buffer = []  # reset
            else:
                # Accumulate description
                buffer.append(line)
    
    # Build DataFrame
    df = pd.DataFrame(rows, columns=["CODE", "NAME", value_column_name])
    
    # Convert numeric values to float
    df[value_column_name] = df[value_column_name].str.replace(".", "", regex=False).str.replace(",", ".", regex=False)
    df[value_column_name] = pd.to_numeric(df[value_column_name], errors="coerce")
    
    return df

In [14]:
def extract_paths(master_list: pd.DataFrame, unique_id: str):
    """
    Extract file paths and page number from master_list for a given unique_id.
    
    Parameters:
        master_list (pd.DataFrame): DataFrame containing metadata.
        unique_id: Index or identifier in the DataFrame.
    
    Returns:
        dict: Dictionary with keys 'pdf_path', 'page_number', 
              'output_pdf_path', 'output_final_path', 'codes_path'.
    """
    return master_list.loc[unique_id, "document_name"], int(master_list.loc[unique_id, "page_number"]), master_list.loc[unique_id, "output_pdf_path"], master_list.loc[unique_id, "output_final_path"], master_list.loc[unique_id, "codes_path"]

In [15]:
def convert_to_dataframe(text: str) -> pd.DataFrame:
    """
    Convert raw AnnexI/Solvency II balance sheet text into a structured DataFrame.

    Parameters
    ----------
    text : str
        Raw text containing balance sheet items.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns: CODE, NAME, VALUE
    """
    rows = []
    
    # Split text into lines
    for line in text.splitlines():
        # Match rows with a code like R001 and a value at the end
        match = re.match(r"^(.*)\s+(R\d+)\s+([-]?\d[\d\.\,]*)$", line.strip())
        if match:
            name = match.group(1).strip()
            code = match.group(2).strip()
            value_str = match.group(3).replace(".", "")  # remove thousand separators
            value = float(value_str.replace(",", "."))   # convert to float
            rows.append((code, name, value))
    
    df = pd.DataFrame(rows, columns=["CODE", "NAME", "VALUE"])
    return df

In [16]:
def append_zero_if_len4(df: pd.DataFrame, col: str = "CODE") -> pd.DataFrame:
    """
    Append '0' to the code if its length is 4 characters.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe containing the code column.
    col : str
        Column name for codes (default = 'CODE').

    Returns
    -------
    pd.DataFrame
        DataFrame with modified codes.
    """
    df = df.copy()
    df[col] = df[col].apply(lambda x: x + "0" if len(x) == 4 else x)
    return df

In [17]:
def run_ocr_and_convert_to_df(path: str, api_key: str)-> pd.DataFrame:
    """
    Run OCR on a PDF file and convert the extracted Markdown table into a DataFrame.
    """
    
    markdown_text = run_mistral_ocr(path, api_key)
    table = markdown_table_to_dataframe(markdown_text)
    return table

In [18]:
def ocr_to_dataframe(ocr_text: str) -> pd.DataFrame:
    """
    Converts OCR extracted financial text into a structured DataFrame.
    
    Extracts description, code (Rxxxx, Cxxxx), and numeric value if present.
    """
    rows = []
    
    # Split text into lines
    for line in ocr_text.splitlines():
        # Look for patterns like "..... R0030 ..... 123.456"
        match = re.match(r"^(.*?)\.{2,}\s*(R\d{4}|C\d{4})(?:\s*\.{2,}\s*([\d\.\s-]+))?$", line.strip())
        if match:
            desc = match.group(1).strip()
            code = match.group(2).strip()
            raw_value = match.group(3)
            
            if raw_value:
                # Remove spaces inside numbers (e.g. "4.275 .590" -> "4.275.590")
                cleaned = raw_value.replace(" ", "")
                # Convert to float if numeric, else keep as string
                try:
                    value = float(cleaned.replace(".", "").replace(",", "."))
                except ValueError:
                    value = cleaned
            else:
                value = None
            
            rows.append((code, desc, value))
    
    df = pd.DataFrame(rows, columns=["CODE", "DESCRIPTION", "VALUE"])
    return df

## The list of companies



In [19]:
master_list = pd.read_csv("master_list.csv", header=0, index_col=0)

In [20]:
display(master_list)

,company,document_name,table_name,page_number,output_pdf_path,output_final_path,codes_path,leto
id,,,,,,,,
CREDEM_VITA_02_1,CREDEM,Input\SFCR 2024 CREDEMVITA.pdf,S.02.01.02_A,197,Single_pdf/CREDEM_S02_01_02_1_2024.pdf,Output/CREDEM_S02_01_02_1_2024.csv,NaN,2024
CREDEM_VITA_02_2,CREDEM,Input\SFCR 2024 CREDEMVITA.pdf,S.02.01.02_L,198,Single_pdf/CREDEM_S02_01_02_2_2024.pdf,Output/CREDEM_S02_01_02_2_2024.csv,NaN,2024
AXA_VITA_02_01,AXA,Input\2024.12 QRT SFCR AXA MPS Assicurazioni V...,S.02.01.02_A,1,Single_pdf/AXA_S02_01_02_1_2024.pdf,Output/AXA_S02_01_02_1_2024.csv,NaN,2024
CREDAG_VITA_02_01,CREDIT_AGRICOLE,Input\ca_vita_sfcr_2024.pdf,S.02.01.02_A,61,Single_pdf/CA_S02_01_02_1_2024.pdf,Output/CA_S02_01_02_1_2024.csv,NaN,2024
CREDAG_VITA_02_02,CREDIT_AGRICOLE,Input\ca_vita_sfcr_2024.pdf,S.02.01.02_L,62,Single_pdf/CA_S02_01_02_2_2024.pdf,Output/CA_S02_01_02_2_2024.csv,NaN,2024
REALE_02_01,REALE_MUTUA,Input\SFCR_A123S_20241231.pdf,S.02.01.02_A,158,Single_pdf/RM_S02_01_02_1_2024.pdf,Output/RM_S02_01_02_1_2024.csv,NaN,2024
REALE_02_02,REALE_MUTUA,Input\SFCR_A123S_20241231.pdf,S.02.01.02_L,159,Single_pdf/RM_S02_01_02_2_2024.pdf,Output/RM_S02_01_02_2_2024.csv,NaN,2024
CARDIF_02_01,CARDIF,Input\SFCR_A421S_20241231.pdf,S.02.01.02_A,106,Single_pdf/CARDIF_S02_01_02_1_2024.pdf,Output/CARDIF_S02_01_02_1_2024.csv,NaN,2024
MEDIO_02_01,MEDIOLANUM,Input\Mediolanum_Vita_Relazione_Unica_2024.pdf,S.02.01.02_A,164,Single_pdf/MEDIO_S02_01_02_1_2024.pdf,Output/MEDIO_S02_01_02_1_2024.csv,NaN,2024


# Code

## Credemvita S.p.A.

S.02.01.02 1

In [21]:
unique_id = "CREDEM_VITA_02_1"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [22]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [23]:
table.columns = ["DESCRIPTION","CODE", "C0010"]
table.drop(index=[0],inplace=True)

In [24]:
set_code_index_and_save(table=table, path=output_final_path)

In [25]:
del table

S.02.01.02 2

In [26]:
unique_id = "CREDEM_VITA_02_2"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [27]:
table = extract_tables_from_pdf(pdf_path=output_pdf_path, page_number=1)

In [28]:
table.columns = ["DESCRIPTION","CODE", "C0010"]
table.drop(index=[39],inplace=True)

In [29]:
set_code_index_and_save(table=table, path=output_final_path)

In [30]:
del table

## AXA MPS Assicurazioni Vita

S.02.01.02 1

In [31]:
unique_id = "AXA_VITA_02_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [32]:
table = extract_tables_from_pdf(pdf_path=output_pdf_path, page_number=1)

Cannot set gray non-stroke color because /'R768' is an invalid float value
Cannot set gray non-stroke color because /'R770' is an invalid float value


In [33]:
table = table.iloc[:,[2,3]]
table.columns = ["CODE", "C0010"]
table.drop(index=[0, 1, 43, 44, 45, 86],inplace=True)

In [34]:
set_code_index_and_save(table=table, path=output_final_path)

In [35]:
del table

## CRÈDIT AGRICOLE VITA

S.02.01.02 1

In [36]:
unique_id = "CREDAG_VITA_02_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [37]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [38]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [39]:
set_code_index_and_save(table=table, path=output_final_path)

In [40]:
del table

S.02.01.02 2

In [41]:
unique_id = "CREDAG_VITA_02_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [42]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [43]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [44]:
set_code_index_and_save(table=table, path=output_final_path)

In [45]:
del table

## Società Reale Mutua di Assicurazioni

S.02.01.02 1

In [46]:
unique_id = "REALE_02_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [47]:
table = extract_tables_from_pdf(pdf_path=output_pdf_path, page_number=1)

In [48]:
table = convert_to_dataframe(table.columns.values[0])

In [49]:
table = append_zero_if_len4(table,"CODE")

In [50]:
table.columns = ["CODE","DESCRIPTION", "C0010"]

In [51]:
set_code_index_and_save(table=table, path=output_final_path)

In [52]:
del table

S.02.01.02 2

In [53]:
unique_id = "REALE_02_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [54]:
table_index = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [55]:
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[page_number - 1]  # Pages are zero-indexed
    tables = page.extract_tables()
    table_numbers = pd.DataFrame(tables[0])

In [56]:
table = pd.concat([table_index, table_numbers], axis=1)

In [57]:
table.iloc[-1,1] = table.iloc[-1,0]

In [58]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [59]:
set_code_index_and_save(table=table, path=output_final_path)

In [60]:
del table
del table_index
del table_numbers

## Cardif Vita S.p.A.

S.02.01.02

In [61]:
unique_id = "CARDIF_02_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [62]:
table = extract_tables_from_pdf(pdf_path=output_pdf_path, page_number=1)

In [63]:
table.columns = ["DESCRIPTION","CODE", "C0010"]
table.drop(index=[0,1,2],inplace=True)

In [64]:
set_code_index_and_save(table=table, path=output_final_path)

In [65]:
del table

## MEDIOLANUM VITA S.p.A.

S.02.01.02 1

In [66]:
unique_id = "MEDIO_02_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [67]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [68]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [69]:
set_code_index_and_save(table=table, path=output_final_path)

In [70]:
del table

S.02.01.02 2

In [71]:
unique_id = "MEDIO_02_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [72]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [73]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [74]:
set_code_index_and_save(table=table, path=output_final_path)

In [75]:
del table

## Generali Italia S.p.A.

S.02.01.02 1

In [76]:
unique_id = "GEN_ITA_02_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [77]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [78]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [79]:
set_code_index_and_save(table=table, path=output_final_path)

In [80]:
del table

S.02.01.02 2

In [81]:
unique_id = "GEN_ITA_02_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [82]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [83]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [84]:
set_code_index_and_save(table=table, path=output_final_path)

In [85]:
del table

## Banco BPM Vita S.p.A.

S.02.01.02 1

In [86]:
unique_id = "BMP_VITA_02_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [87]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [88]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [89]:
set_code_index_and_save(table=table, path=output_final_path)

In [90]:
del table

S.02.01.02 2

In [91]:
unique_id = "BMP_VITA_02_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [92]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [93]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [94]:
set_code_index_and_save(table=table, path=output_final_path)

In [95]:
del table

## HDI ASSICURAZIONI S.p.A.

S.02.01.02 1

In [96]:
unique_id = "HDI_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [97]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [98]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [99]:
set_code_index_and_save(table=table, path=output_final_path)

In [100]:
del table

S.02.01.02 2

In [101]:
unique_id = "HDI_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [102]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [103]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [104]:
set_code_index_and_save(table=table, path=output_final_path)

In [105]:
del table

## Gruppo Assicurativo Poste Vita

S.02.01.02 1

In [106]:
unique_id = "POSTE_VITA_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [107]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [108]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [109]:
set_code_index_and_save(table=table, path=output_final_path)

In [110]:
del table

S.02.01.02 2

In [111]:
unique_id = "POSTE_VITA_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [112]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [113]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [114]:
set_code_index_and_save(table=table, path=output_final_path)

In [115]:
del table

## FIDEURAM VITA S.P.A.

S.02.01.02 1

In [116]:
unique_id = "INTESA_VITA_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [117]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [118]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [119]:
set_code_index_and_save(table=table, path=output_final_path)

In [120]:
del table

S.02.01.02 2

In [121]:
unique_id = "INTESA_VITA_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [122]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [123]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [124]:
set_code_index_and_save(table=table, path=output_final_path)

In [125]:
del table

## CNP Vita Assicura S.p.A.

S.02.01.02 1

In [126]:
unique_id = "CNP_VITA_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [127]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [128]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [129]:
set_code_index_and_save(table=table, path=output_final_path)

In [130]:
del table

S.02.01.02 2

In [131]:
unique_id = "CNP_VITA_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [132]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [133]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [134]:
set_code_index_and_save(table=table, path=output_final_path)

In [135]:
del table

## ITAS VITA

S.02.01.02 1

In [136]:
unique_id = "ITAS_VITA_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [137]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [138]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [139]:
set_code_index_and_save(table=table, path=output_final_path)

In [140]:
del table

S.02.01.02 2

In [141]:
unique_id = "ITAS_VITA_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [142]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [143]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [144]:
set_code_index_and_save(table=table, path=output_final_path)

In [145]:
del table

## Helvetia Vita S.p.A.

S.02.01.02 1

In [146]:
unique_id = "HELVETIA_VITA_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [147]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [148]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [149]:
set_code_index_and_save(table=table, path=output_final_path)

In [150]:
del table

S.02.01.02 2

In [151]:
unique_id = "HELVETIA_VITA_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [152]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [153]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [154]:
set_code_index_and_save(table=table, path=output_final_path)

In [155]:
del table

## Vittoria Assicurazioni S.p.A.

S.02.01.02 1

In [156]:
unique_id = "VITTORIA_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [157]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [158]:
table.columns = ["CODE", "C0010"] # ONE COLUMN LESS

In [159]:
set_code_index_and_save(table=table, path=output_final_path)

In [160]:
del table

S.02.01.02 2

In [161]:
unique_id = "VITTORIA_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [162]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [163]:
table.columns = ["CODE", "C0010"] # ONE COLUMN LESS

In [164]:
set_code_index_and_save(table=table, path=output_final_path)

In [165]:
del table

## GROUPAMA ASSICURAZIONI S.P.A.

S.02.01.02

In [166]:
unique_id = "GROUPAMA_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [167]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [168]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [169]:
set_code_index_and_save(table=table, path=output_final_path)

In [170]:
del table

## UniCredit Allianz Vita S.p.A.

S.02.01.02 1

In [171]:
unique_id = "ALLIANZ_UNICREDIT_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [172]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [173]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [174]:
set_code_index_and_save(table=table, path=output_final_path)

In [175]:
del table

S.02.01.02 2

In [176]:
unique_id = "ALLIANZ_UNICREDIT_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [177]:
table = run_ocr_and_convert_to_df(path=output_pdf_path, api_key=api_key)

In [178]:
table.columns = ["DESCRIPTION","CODE", "C0010"]

In [179]:
set_code_index_and_save(table=table, path=output_final_path)

In [180]:
del table

## Zurich Investments Life S.p.A.

S.02.01.02 1 

In [181]:
unique_id = "ZURICH_LIFE_01"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [182]:
markdown_text = run_mistral_ocr(output_pdf_path, api_key)
table = ocr_to_dataframe(markdown_text)

In [183]:
table.columns = ["CODE", "DESCRIPTION", "C0010"]

In [184]:
set_code_index_and_save(table=table, path=output_final_path)

In [185]:
del table

S.02.01.02 2

In [186]:
unique_id = "ZURICH_LIFE_02"
pdf_path, page_number, output_pdf_path, output_final_path, codes_path =  extract_paths(master_list=master_list, unique_id=unique_id)
extract_page(input_pdf_path=pdf_path, output_pdf_path=output_pdf_path, page_number=page_number, password = "")

In [187]:
markdown_text = run_mistral_ocr(output_pdf_path, api_key)
table = ocr_to_dataframe(markdown_text)

In [188]:
table.columns = ["CODE", "DESCRIPTION", "C0010"]

In [189]:
set_code_index_and_save(table=table, path=output_final_path)

In [190]:
del table